# Helix

In [1]:
import numpy as np


def circular_helix(a=None, b=None, *args, slope=None, pitch=None):
    """A circular helix of radius a and slope a/b (or pitch 2πb)"""
    if pitch is not None:
        b = b if b is not None else pitch / 2 / np.pi
    if slope is not None:
        a = a if a is not None else slope * b
        b = b if b is not None else slope / a
    def inner(t):
        return a * np.cos(t), a * np.sin(t), b * t
    return inner


circular_helix(1, 5)(1)

(0.5403023058681398, 0.8414709848078965, 5)

In [2]:
from polymesh.space import CartesianFrame
from polymesh import PolyData, LineData
import numpy as np

frame = CartesianFrame(dim=3)

coords = np.array(list(map(circular_helix(5, 2), np.linspace(0, 30, 100))))
topo = np.zeros((coords.shape[0]-1, 2))
topo[:, 0] = np.arange(topo.shape[0])
topo[:, 1] = topo[:, 0] + 1

mesh = PolyData(coords=coords, frame=frame)
mesh['helix'] = LineData(topo=topo, frame=frame)

mesh.plot(notebook=False)

In [ ]:
from sigmaepsilon.solid import Structure, LineMesh, PointData
from neumann.linalg import linspace, Vector
from polymesh.space import StandardFrame, PointCloud, frames_of_lines
from sigmaepsilon.solid.fem.cells import B2 as Beam

import numpy as np
from numpy import pi as PI

# INPUT DATA
w, h = 10., 10.  # width and height of the rectangular cross section
F = -100.  # value of the vertical load at the free end
E = 210000.0  # Young's modulus
nu = 0.3  # Poisson's ratio

# cross section
A = w * h  # area
Iy = w * h**3 / 12  # second moment of inertia around the y axis
Iz = w * h**3 / 12  # second moment of inertia around the z axis
Ix = Iy + Iz  # torsional inertia

# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

# space
GlobalFrame = StandardFrame(dim=3)

# support at the first, load at the last node
loads = np.zeros((coords.shape[0], 6))
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
global_load_vector = Vector([0., 0, -1], frame=GlobalFrame).show()
loads[-1, :3] = global_load_vector
fixity[0, :] = True

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = frames_of_lines(coords, topo)
cd = Beam(topo=topo, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)
structure.linsolve()

# postproc
# 1) displace the mesh
dofsol = structure.nodal_dof_solution()[:, :3]
